<a href="https://colab.research.google.com/github/patrickctrf/IA024_2022S2/blob/main/ex07/patrick_ferreira/ex07_patrick_ferreira_175480.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook de referência 

Nome: 

## Instruções

Neste colab iremos treinar um modelo T5 para traduzir de inglês para português. Iremos treiná-lo com o data Paracrawl.

- Usaremos o dataset Paracrawl Inglês-Português. Truncamos o dataset de treino para apenas 100k pares para deixar o treinamento mais rápido. Quem quiser pode treinar com mais amostras. Se demorar muito para treinar, truncar o dataset ainda mais.

- Usaremos o BLEU como métrica. Usaremos o SacreBLEU pois sempre faz o mesmo pré-processamento (tokenização, lowercase). Não usaremos torchnlp.metrics.bleu, torchtext.data.metrics.bleu_score, etc. SacreBLEU é lento: usar poucas amostras de validação (ex: 5k)


Usaremos o modelo PTT5 disponível no model hub da HuggingFace:

https://huggingface.co/unicamp-dl/ptt5-small-portuguese-vocab

Este é  um T5 pré-treinado em textos em português e com tokenizador em português.

É recomendável salvar os pesos do modelo e estado dos otimizadores, pois o treinamento é longo.


In [10]:
# Configurações gerais
from asyncio import Queue
from threading import Thread

import numpy as np
from torch import nn
from tqdm import tqdm

model_name = "unicamp-dl/ptt5-small-portuguese-vocab"
batch_size = 64
accumulate_grad_batches = 2
source_max_length = 128
target_max_length = 128
learning_rate = 1e-3

In [11]:
! pip install sacrebleu
! pip install transformers
! pip install sentencepiece

In [12]:
# Importar todos os pacotes de uma só vez para evitar duplicados ao longo do notebook.
import gzip
import os
import random
import sacrebleu
import torch
import torch.nn.functional as F

# from google.colab import drive

from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from typing import Dict
from typing import List
from typing import Tuple

In [13]:
# Important: Fix seeds so we can replicate results
seed = 123
random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)

if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


Iremos salvar os checkpoints (pesos do modelo) no google drive, para que possamos continuar o treino de onde paramos.

In [14]:
# drive.mount('/content/drive')

## Preparando Dados

Primeiro, fazemos download do dataset:

In [15]:
! wget -nc https://storage.googleapis.com/unicamp-dl/ia024a_2022s2/paracrawl_enpt_train.tsv.gz
! wget -nc https://storage.googleapis.com/unicamp-dl/ia024a_2022s2/paracrawl_enpt_test.tsv.gz

File ‘paracrawl_enpt_train.tsv.gz’ already there; not retrieving.

File ‘paracrawl_enpt_test.tsv.gz’ already there; not retrieving.



## Carregando o dataset

Criaremos uma divisão de treino (100k pares) e val (5k pares) artificialmente.

Nota: Evitar de olhar ao máximo o dataset de teste para não ficar enviseado no que será testado. Em aplicações reais, o dataset de teste só estará disponível no futuro, ou seja, é quando o usuário começa a testar o seu produto.

In [16]:
def load_text_pairs(path):
    text_pairs = []
    for line in gzip.open(path, mode='rt'):
        text_pairs.append(line.strip().split('\t'))
    return text_pairs

x_train = load_text_pairs('paracrawl_enpt_train.tsv.gz')
x_test = load_text_pairs('paracrawl_enpt_test.tsv.gz')

# Embaralhamos o treino para depois fazermos a divisão treino/val.
random.shuffle(x_train)

# Truncamos o dataset para 100k pares de treino e 5k pares de validação.
x_val = x_train[100000:105000]
x_train = x_train[:100000]

for set_name, x in [('treino', x_train), ('validação', x_val), ('test', x_test)]:
    print(f'\n{len(x)} amostras de {set_name}')
    print(f'3 primeiras amostras {set_name}:')
    for i, (source, target) in enumerate(x[:3]):
        print(f'{i}: source: {source}\n   target: {target}')


100000 amostras de treino
3 primeiras amostras treino:
0: source: More Croatian words and phrases
   target: Mais palavras e frases em croata
1: source: Jerseys and pullovers, containing at least 50Â % by weight of wool and weighing 600Â g or more per article 6110 11 10 (PCE)
   target: Camisolas e pulôveres, com pelo menos 50 %, em peso, de lã e pesando 600g ou mais por unidade 6110 11 10 (PCE)
2: source: Atex Colombia SAS makes available its lead product, 100% natural liquid latex, excellent quality and price. ... Welding manizales caldas Colombia a DuckDuckGo
   target: Atex Colômbia SAS torna principal produto está disponível, látex líquido 100% natural, excelente qualidade e preço. ...

5000 amostras de validação
3 primeiras amostras validação:
0: source: «You have hidden these things from the wise and the learned you have revealed them to the childlike»
   target: «Escondeste estas coisas aos sábios e entendidos e as revelaste aos pequenos»
1: source: Repair of computers, applic

Criando Dataset


In [17]:
tokenizer = T5Tokenizer.from_pretrained(model_name)

In [20]:
class MyDataset(Dataset):
    def __init__(self, text_pairs: List[Tuple[str]], tokenizer,
                 source_max_length: int = 32, target_max_length: int = 32):
        self.tokenizer = tokenizer
        self.text_pairs = text_pairs
        self.source_max_length = source_max_length
        self.target_max_length = target_max_length

        self.sources, self.targets = list(zip(*text_pairs))
        
    def __len__(self):
        return len(self.text_pairs)
    
    def __getitem__(self, idx):
        # source, target = self.text_pairs[idx]
        # TODO: tokenizar texto

        source_token_ids =  tokenizer(self.sources, padding=True, truncation=True, max_length=self.source_max_length, return_tensors = "pt").input_ids
        source_mask =       tokenizer(self.sources, padding=True, truncation=True, max_length=self.source_max_length, return_tensors = "pt").attention_mask
        target_token_ids =  tokenizer(self.targets, padding=True, truncation=True, max_length=self.source_max_length, return_tensors = "pt").input_ids
        target_mask =       tokenizer(self.targets, padding=True, truncation=True, max_length=self.source_max_length, return_tensors = "pt").attention_mask

        return source_token_ids, source_mask, target_token_ids, target_mask, self.sources[idx], self.targets[idx]

## Testando o DataLoader

In [21]:
text_pairs = [('we like pizza', 'eu gosto de pizza')]
dataset_debug = MyDataset(
    text_pairs=text_pairs,
    tokenizer=tokenizer,
    source_max_length=source_max_length,
    target_max_length=target_max_length)

dataloader_debug = DataLoader(dataset_debug, batch_size=10, shuffle=True, 
                              num_workers=0)

source_token_ids, source_mask, target_token_ids, target_mask, _, _ = next(iter(dataloader_debug))
print('source_token_ids:\n', source_token_ids)
print('source_mask:\n', source_mask)
print('target_token_ids:\n', target_token_ids)
print('target_mask:\n', target_mask)

print('source_token_ids.shape:', source_token_ids.shape)
print('source_mask.shape:', source_mask.shape)
print('target_token_ids.shape:', target_token_ids.shape)
print('target_mask.shape:', target_mask.shape)

source_token_ids:
 tensor([[[  31, 1528, 1079,  634, 1241, 7531,    1]]])
source_mask:
 tensor([[[1, 1, 1, 1, 1, 1, 1]]])
target_token_ids:
 tensor([[[2077, 6618,    4, 1241, 7531,    1]]])
target_mask:
 tensor([[[1, 1, 1, 1, 1, 1]]])
source_token_ids.shape: torch.Size([1, 1, 7])
source_mask.shape: torch.Size([1, 1, 7])
target_token_ids.shape: torch.Size([1, 1, 6])
target_mask.shape: torch.Size([1, 1, 6])


## Criando DataLoaders de Treino/Val/Test

In [22]:
dataset_train = MyDataset(text_pairs=x_train,
                          tokenizer=tokenizer,
                          source_max_length=source_max_length,
                          target_max_length=target_max_length)

dataset_val = MyDataset(text_pairs=x_val,
                        tokenizer=tokenizer,
                        source_max_length=source_max_length,
                        target_max_length=target_max_length)

dataset_test = MyDataset(text_pairs=x_test,
                         tokenizer=tokenizer,
                         source_max_length=source_max_length,
                         target_max_length=target_max_length)

train_dataloader = DataLoader(dataset_train, batch_size=batch_size,
                              shuffle=True, num_workers=0)

val_dataloader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, 
                            num_workers=0)

test_dataloader = DataLoader(dataset_test, batch_size=batch_size,
                             shuffle=False, num_workers=0)

### Utilidade para converter dados de device em paralelo

In [ ]:
class DataManager(Thread):
    def __init__(self, data_loader, buffer_size=3, device=torch.device("cpu"), data_type=torch.float32):
        """
This manager intends to load a PyTorch dataloader like from disk into memory,
reducing the acess time. It does not easily overflow memory, because we set a
buffer size limiting how many samples will be loaded at once. Everytime a sample
is consumed by the calling thread, another one is replaced in the
buffer (unless we reach the end of dataloader).

A manger may be called exactly like a dataloader, an it's based in an internal
thread that loads samples into memory in parallel. This is specially useful
when you are training in GPU and processor is almost idle.

        :param data_loader: Base dataloader to load in parallel.
        :param buffer_size: How many samples to keep loaded (caution to not overflow RAM). Must be integer > 0. Default: 3.
        :param device: Torch device to put samples in, like torch.device("cpu") (default). It saves time by transfering in parallel.
        :param data_type: Automatically casts tensor type. Default: torch.float32.
        """
        super().__init__()
        self.buffer_queue = Queue(maxsize=buffer_size)
        self.data_loader = data_loader
        self.buffer_size = buffer_size
        self.device = device
        self.data_type = data_type

        self.dataloader_finished = False

    def run(self):
        """
Runs the internal thread that iterates over the dataloader until fulfilling the
buffer or the end of samples.
        """
        for i, sample in enumerate(self.data_loader):
            # Important to set before put in queue to avoid race condition
            # would happen if trying to get() in next() method before setting this flag
            if i >= len(self) - 1:
                self.dataloader_finished = True

            self.buffer_queue.put([x.to(dtype=self.data_type, device=self.device, non_blocking=True) for x in sample])

    def __iter__(self):
        """
Returns an iterable of itself.

        :return: Iterable around this class.
        """
        self.start()
        self.dataloader_finished = False
        return self

    def __next__(self):
        """
Intended to be used as iterator.

        :return: Next iteration element.
        """
        if self.dataloader_finished is True and self.buffer_queue.empty():
            raise StopIteration()

        return self.buffer_queue.get()

    def __len__(self):
        return len(self.data_loader)

### TREINAMENTO

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import GradScaler, autocast

model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

max_examples = 100_000
eval_every_steps = 1000
lr = 1e-3
use_amp = True

train_loader = DataManager(train_dataloader, device=device, data_type=None)
validation_loader = DataManager(val_dataloader, device=device, data_type=None)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.9, min_lr=3e-5, patience=15000, threshold=1e-1, verbose=True)
scaler=GradScaler()


def train_step(source_tokens, source_mask, target_tokens, target_mask):
    model.train()
    model.zero_grad()
    with autocast(enabled=use_amp):
        loss = model(input_ids = source_tokens, attention_mask = source_mask, decoder_attention_mask = target_mask, labels = target_tokens).loss
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    return loss.item()


def validation_step(source_tokens, source_mask, target_tokens, target_mask):
    model.eval()
    with torch.no_grad:
        with autocast(enabled=use_amp):
            loss = model(input_ids = source_tokens, attention_mask = source_mask, decoder_attention_mask = target_mask, labels = target_tokens).loss
    return loss.item()


best_validation_loss = 9999
train_losses = []
n_examples = 0
step = 0
pbar = tqdm(total=max_examples)
while n_examples < max_examples:
    for mini_batch in train_loader:
        loss = train_step(*mini_batch)
        train_losses.append(loss)

        # LR scheduler
        scheduler.step(loss)

        if step % eval_every_steps == 0:
            train_loss = np.average(train_losses)

            with torch.no_grad():
                valid_loss = np.average([
                    validation_step(*mini_batch)
                    for mini_batch in validation_loader])
                # Checkpoint to best models found.
                if best_validation_loss > valid_loss:
                    # Update the new best perplexity.
                    best_validation_loss = valid_loss
                    model.eval()
                    torch.save(model, "best_model.pth")

            print(f'{step} steps; {n_examples} examples so far; train loss: {train_loss:.2f}, valid loss: {valid_loss:.2f}')
            train_losses = []

        n_examples += mini_batch[0].shape[0]  # Increment of batch size
        step += 1
        pbar.update(mini_batch[0].shape[0])
        if n_examples >= max_examples:
            break

pbar.close()

# Restore best model (checkpoint) found
model = torch.load("best_model.pth")

